In [1]:
import numpy as np
import pandas as pd
import os
import math
import time
import torch
from data_preprocessor import *
import argparse
import torch.utils.data as Data
import torch.optim as optim


In [2]:
class Arg_class():
    def __init__(self):
        self.hidden_neuron = 500
        self.lambda_value = 1.0
        self.train_epoch = 2000
        self.batch_size = 100
        self.optimizer_method = "Adam"
        self.grad_clip = False
        self.base_lr = 1e-3
        self.decay_epoch_step = 50
        self.random_seed = 1000
        self.display_step = 1
        self.weight_decay = 1e-4
        
args = Arg_class()


In [3]:
class AutoRecModel(torch.nn.Module):
    def __init__(self,args,
                      num_users,num_items,
                      ):
        super(AutoRecModel,self).__init__()
        
        self.num_users = num_users
        self.num_items = num_items

        self.hidden_neuron = args.hidden_neuron
        self.lambda_value = args.lambda_value
        self.encoder = torch.nn.Linear(self.num_items,self.hidden_neuron)
        self.decoder = torch.nn.Linear(self.hidden_neuron,self.num_items)
        
    def forward(self,x): #shape batch_size,num_items
        x = self.encoder(x)
#         x = torch.relu(x)
        x = torch.sigmoid(x)
        x = self.decoder(x)
        return x
    
    def loss_function(self,predicted_rating,true_rating, mask_input,optimizer):
        loss = 0
        temp = 0
        loss += ((predicted_rating - true_rating) * mask_input).pow(2).sum() #mask input 只有有值的进行预测
        rmse = loss
        for i in optimizer.param_groups:
            for j in i['params']:
                if j.data.dim() == 2:
                    temp += torch.t(j.data).pow(2).sum()
        loss += temp * self.lambda_value * 0.5
        return loss,rmse
    
def train(epoch):
    RMSE = 0
    loss_all = 0
    for step, (batch_x,batch_mask_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.type(torch.FloatTensor).cuda()
        batch_mask_x = batch_mask_x.type(torch.FloatTensor).cuda()
        predicted_rating = model(batch_x)
        loss,rmse = model.loss_function(predicted_rating, batch_x,batch_mask_x,optimizer)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_all += loss
        RMSE += rmse
        
    RMSE = np.sqrt(RMSE.detach().cpu().numpy() / (train_mask_r == 1).sum())
    if epoch%10 == 0:
        print("epoch:{}, train_RMSE: {}".format(epoch,RMSE))

def test(epoch):
    test_x = torch.from_numpy(test_r).type(torch.FloatTensor).cuda()
    test_mask_x = torch.from_numpy(test_mask_r).type(torch.FloatTensor).cuda()
    
    predicted_rating = model(test_x)
    
    #未在训练集里出现过的
    unseen_user_test_list = list(user_test_set - user_train_set)
    unseen_item_test_list = list(item_test_set - item_train_set)
    
    for user in unseen_user_test_list:
        for item in unseen_item_test_list:
            if test_mask_x[user,item] == 1:
                predicted_rating[user,item] = 3
    mse = ((predicted_rating - test_x) * test_mask_x).pow(2).sum()
    RMSE = mse.detach().cpu().numpy() / (test_mask_r == 1).sum()
    RMSE = np.sqrt(RMSE)
    if epoch%10 == 0:
        print("epoch:{}, test_RMSE: {}".format(epoch,RMSE))
    

In [4]:
data_name = 'ml-1m'
num_users = 6040
num_items = 3952
num_total_ratings = 1000209
train_ratio = 0.9
path = "./data/%s" % data_name + "/"
train_r, train_mask_r, C, train_R, train_mask_R, test_r, test_mask_r,num_train_ratings,num_test_ratings,\
user_train_set,item_train_set,user_test_set,item_test_set = read_rating(path,num_users,num_items,\
                                                                       num_total_ratings,1,0,train_ratio)

In [5]:
model = AutoRecModel(args,num_users,num_items)
model.cuda()

AutoRecModel(
  (encoder): Linear(in_features=3952, out_features=500, bias=True)
  (decoder): Linear(in_features=500, out_features=3952, bias=True)
)

In [6]:
optimizer = optim.Adam(model.parameters(),lr = args.base_lr,weight_decay=args.weight_decay)

In [7]:
train_dataset = Data.TensorDataset(torch.from_numpy(train_r), torch.from_numpy(train_mask_r),\
                                  torch.from_numpy(train_r))
train_loader = Data.DataLoader(dataset = train_dataset,
                              batch_size = args.batch_size,
                              shuffle = True)

In [8]:
for ep in range(args.train_epoch):
    train(ep)
    test(ep)

epoch:0, train_RMSE: 1.4563815438047005
epoch:0, test_RMSE: 1.2451797162536773
epoch:10, train_RMSE: 0.9153742755269901
epoch:10, test_RMSE: 1.0449873170601203
epoch:20, train_RMSE: 0.8246099889272066
epoch:20, test_RMSE: 0.9001798366243072
epoch:30, train_RMSE: 0.809775517408858
epoch:30, test_RMSE: 0.8308364809069344
epoch:40, train_RMSE: 0.8412279926464984
epoch:40, test_RMSE: 0.8063018137619908
epoch:50, train_RMSE: 0.8788197820032665
epoch:50, test_RMSE: 0.8006776799705906
epoch:60, train_RMSE: 0.9124406131330677
epoch:60, test_RMSE: 0.8022074609443727
epoch:70, train_RMSE: 0.9417037260016119
epoch:70, test_RMSE: 0.8045483283991253
epoch:80, train_RMSE: 0.9653816447397993
epoch:80, test_RMSE: 0.8100199802798848
epoch:90, train_RMSE: 0.9851240007821068
epoch:90, test_RMSE: 0.8155698288646973
epoch:100, train_RMSE: 1.0030343861016144
epoch:100, test_RMSE: 0.8183588195479525
epoch:110, train_RMSE: 1.0188797015782216
epoch:110, test_RMSE: 0.8223746630076009
epoch:120, train_RMSE: 1.03

KeyboardInterrupt: 